In [361]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from prettytable import PrettyTable
from datetime import datetime, timedelta
from tqdm import tqdm
import statsmodels.api as sm
import yfinance as yf
import numpy as np


In [362]:
IPCA=4
SELIC=12.75
ANO_DIAS_UTEIS=252

In [371]:
tickers=["DIVO11.SA","^BVSP"]
data = yf.download(tickers, start="2018-1-1")["Adj Close"]
data

[*********************100%%**********************]  2 of 2 completed


,DIVO11.SA,^BVSP
Date,,
2018-01-02,41.820000,77891.000000
2018-01-03,41.880001,77995.000000
2018-01-04,42.310001,78647.000000
2018-01-05,42.450001,79071.000000
2018-01-08,42.450001,79379.000000
...,...,...
2023-10-03,79.000000,113419.000000
2023-10-04,78.669998,113607.000000
2023-10-05,78.339996,113284.000000


In [364]:
# beta_alpha 360
def alpha_beta(data, ticker, indice):
   
    #Calcular os retornos logarítmicos
    retorno_ativos = np.log(data/data.shift(1))

    # Remover os NaNs
    retorno_ativos = retorno_ativos.dropna()

    # Definir y (ticker) e X (indice)
    y = retorno_ativos[tickers[0]]
    X = retorno_ativos[tickers[1]]

    # Adicionar uma constante ao X (para o intercepto)
    X = sm.add_constant(X)

    # Realizar a regressão
    model = sm.OLS(y,X).fit()

    # Coletar o alpha e beta
    alpha = model.params[0]
    beta = model.params[1]
    
    return alpha,beta


a,b = alpha_beta(data,tickers[0],tickers[1])
b,a

(0.7738897740482866, 0.0002468718165196509)

In [370]:
def retorno_esperado(beta,taxa_livre_risco, premio_risco):
    re = taxa_livre_risco + beta * (premio_risco-taxa_livre_risco)
    return  re

selic=12/100retorno esperado
ipca=(4+2)/100
re = retorno_esperado(b,selic,ipca)
re

0.0735666135571028

In [366]:
def sharpe_ratio(data, ticker, retorno_esperado, taxa_livre_risco):
     
    daily_returns = np.log(data/data.shift(1))
    daily_returns = daily_returns[ticker].dropna()
    std = daily_returns.std() * (252**0.5)

    
    # Calcula o Sharpe Ratio
    sharpe = (retorno_esperado - taxa_livre_risco) / std
    
    return sharpe

sharpe_ratio(data,tickers[0],re,0.12)

-0.20346213069517832

In [367]:
def sortino_ratio(data, ticker, retorno_esperado, taxa_livre_risco):
     
    daily_returns = np.log(data/data.shift(1))
    daily_returns = daily_returns[ticker].dropna()
    downside = daily_returns[daily_returns < 0]
    std = downside.std() * (252**0.5)
    
    # Calcula o Sharpe Ratio
    sortino = (retorno_esperado - taxa_livre_risco) / std
    
    return sortino

sortino_ratio(data,tickers[0],re,0.12)

-0.25671860267445873